In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime, timedelta

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from app.comet.mcp_client import MCPClient
from app.comet.prompt_manager import PromptManager

%matplotlib inline

## 1. Initialize Comet AI with Prompt Manager

In [ ]:
# Initialize clients
comet = MCPClient()
pm = PromptManager()

print("✓ Comet AI initialized")
print("✓ Prompt Manager initialized")

## 2. Load Sample Signal Data

In production, this would come from your signal generation system.

In [ ]:
# Example signals (replace with actual signal loading)
sample_signals = [
    {
        'symbol': 'NSE:NIFTY50-INDEX',
        'signal_type': 'BULLISH',
        'entry_price': 22000,
        'current_price': 22050,
        'timeframe': '15m',
        'indicators': 'RSI: 65, MACD: Bullish Crossover, Volume: Above Average',
        'market_context': 'Strong uptrend, above all EMAs, positive momentum'
    },
    {
        'symbol': 'NSE:BANKNIFTY-INDEX',
        'signal_type': 'BEARISH',
        'entry_price': 45000,
        'current_price': 44900,
        'timeframe': '5m',
        'indicators': 'RSI: 72 (Overbought), MACD: Bearish Divergence, Volume: Decreasing',
        'market_context': 'Resistance rejection, weakening momentum'
    },
    {
        'symbol': 'NSE:RELIANCE-EQ',
        'signal_type': 'BULLISH',
        'entry_price': 2450,
        'current_price': 2455,
        'timeframe': '1h',
        'indicators': 'RSI: 58, MACD: Neutral, Volume: Average',
        'market_context': 'Consolidating near ATH, awaiting breakout'
    }
]

signals_df = pd.DataFrame(sample_signals)
print(f"Loaded {len(signals_df)} signals for validation\n")
signals_df

## 3. Validate Signals Using Comet AI Templates

In [ ]:
# Validate each signal using the signal_analysis template
validation_results = []

for idx, signal in signals_df.iterrows():
    print(f"\n{'='*60}")
    print(f"Validating Signal #{idx+1}: {signal['symbol']} ({signal['signal_type']})")
    print(f"{'='*60}")
    
    # Format prompt using template
    prompt = pm.format_prompt(
        "signal_analysis",
        symbol=signal['symbol'],
        signal_type=signal['signal_type'],
        entry_price=signal['entry_price'],
        current_price=signal['current_price'],
        timeframe=signal['timeframe'],
        indicators=signal['indicators'],
        market_context=signal['market_context']
    )
    
    try:
        # Get Comet AI analysis
        response = comet.query(prompt)
        analysis = response.get('content', 'No analysis available')
        
        print(f"\n📊 Comet AI Analysis:\n{analysis}\n")
        
        validation_results.append({
            'signal_idx': idx,
            'symbol': signal['symbol'],
            'signal_type': signal['signal_type'],
            'comet_analysis': analysis,
            'validated': True
        })
        
    except Exception as e:
        print(f"❌ Error validating signal: {e}")
        validation_results.append({
            'signal_idx': idx,
            'symbol': signal['symbol'],
            'signal_type': signal['signal_type'],
            'comet_analysis': f'Error: {str(e)}',
            'validated': False
        })

validation_df = pd.DataFrame(validation_results)
print(f"\n✅ Validated {len(validation_df)} signals")

## 4. Risk Assessment for Signals

In [ ]:
# Example: Assess risk for the first signal
signal = signals_df.iloc[0]

print(f"\n🎯 Risk Assessment for {signal['symbol']}\n")

# Format risk assessment prompt
risk_prompt = pm.format_prompt(
    "risk_assessment",
    symbol=signal['symbol'],
    position_type='LONG' if signal['signal_type'] == 'BULLISH' else 'SHORT',
    entry_price=signal['entry_price'],
    current_price=signal['current_price'],
    position_size=50,  # Example position size
    unrealized_pnl=0,  # New position
    total_capital=1000000,
    position_percentage=5,
    current_drawdown=2,
    open_positions_count=3,
    market_conditions=signal['market_context'],
    technical_indicators=signal['indicators']
)

try:
    risk_response = comet.query(risk_prompt)
    print(risk_response.get('content', 'No risk assessment available'))
except Exception as e:
    print(f"❌ Error in risk assessment: {e}")

## 5. Generate Trade Plan

In [ ]:
# Generate complete trade plan for a signal
signal = signals_df.iloc[0]

print(f"\n📋 Generating Trade Plan for {signal['symbol']}\n")

trade_plan_prompt = pm.format_prompt(
    "trade_plan",
    symbol=signal['symbol'],
    strategy_name="Volume Rocket",
    signal_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    signal_type=signal['signal_type'],
    entry_price=signal['entry_price'],
    current_price=signal['current_price'],
    entry_confidence=8,  # Example confidence score
    technical_setup=signal['indicators'],
    max_risk_percent=2,
    account_size=1000000,
    max_position_size=50000,
    market_conditions=signal['market_context']
)

try:
    plan_response = comet.query(trade_plan_prompt)
    print(plan_response.get('content', 'No trade plan available'))
except Exception as e:
    print(f"❌ Error generating trade plan: {e}")

## 6. Market Context Analysis

In [ ]:
# Get broader market context
print("\n🌍 Market Context Analysis\n")

market_prompt = pm.format_prompt(
    "market_context",
    symbol="NSE:NIFTY50-INDEX",
    current_price=22000,
    day_high=22100,
    day_low=21950,
    volume=5000000,
    avg_volume=3500000,
    atr=150,
    index_performance="NIFTY: +0.8%, SENSEX: +0.6%, BANKNIFTY: +1.2%",
    recent_events="US Fed comments positive, FII buying continues",
    technical_context="Consolidating near ATH, strong support at 21800",
    current_time=datetime.now().strftime("%I:%M %p IST"),
    market_session="Pre-noon" if datetime.now().hour < 12 else "Post-noon",
    day_of_week=datetime.now().strftime("%A")
)

try:
    market_response = comet.query(market_prompt)
    print(market_response.get('content', 'No market analysis available'))
except Exception as e:
    print(f"❌ Error in market analysis: {e}")

## 7. Build Signal Confidence Scoring

Based on Comet AI analysis, assign confidence scores to signals.

In [ ]:
# Extract confidence scores from Comet analysis
# This is a simplified example - in production, parse structured responses

def extract_confidence_score(analysis_text):
    """Extract confidence score from Comet analysis (simplified)"""
    # Look for phrases like "Signal Strength: 8/10" or "confidence: 7"
    # This is a placeholder - implement proper parsing
    import re
    
    # Try to find rating pattern
    patterns = [
        r'(\d+)/10',
        r'strength.*?(\d+)',
        r'rating.*?(\d+)',
        r'score.*?(\d+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, analysis_text.lower())
        if match:
            return int(match.group(1))
    
    return 5  # Default medium confidence

# Add confidence scores
validation_df['confidence_score'] = validation_df['comet_analysis'].apply(extract_confidence_score)

print("\n📊 Signal Confidence Scores:\n")
print(validation_df[['symbol', 'signal_type', 'confidence_score']])

# Visualization
plt.figure(figsize=(10, 6))
plt.bar(validation_df['symbol'], validation_df['confidence_score'])
plt.axhline(y=7, color='g', linestyle='--', label='High Confidence Threshold')
plt.axhline(y=4, color='r', linestyle='--', label='Low Confidence Threshold')
plt.xlabel('Symbol')
plt.ylabel('Confidence Score')
plt.title('Signal Confidence Scores')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Decision Framework

Based on Comet AI validation, decide which signals to trade.

In [ ]:
# Define decision thresholds
HIGH_CONFIDENCE = 7
MEDIUM_CONFIDENCE = 5

def make_trading_decision(row):
    """Decide whether to trade based on confidence score"""
    score = row['confidence_score']
    
    if score >= HIGH_CONFIDENCE:
        return 'EXECUTE - High Confidence'
    elif score >= MEDIUM_CONFIDENCE:
        return 'EXECUTE - Medium Confidence (Reduce Size)'
    else:
        return 'SKIP - Low Confidence'

validation_df['decision'] = validation_df.apply(make_trading_decision, axis=1)

print("\n✅ Trading Decisions:\n")
print(validation_df[['symbol', 'signal_type', 'confidence_score', 'decision']])

# Summary
execute_count = validation_df['decision'].str.contains('EXECUTE').sum()
skip_count = validation_df['decision'].str.contains('SKIP').sum()

print(f"\n📈 Summary:")
print(f"   Execute: {execute_count} signals")
print(f"   Skip: {skip_count} signals")
print(f"   Total Validated: {len(validation_df)} signals")

## 9. Export Validation Results

In [ ]:
# Save validation results
output_file = f'signal_validation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
validation_df.to_csv(output_file, index=False)

print(f"\n✅ Validation results saved to: {output_file}")
print(f"\nColumns saved: {list(validation_df.columns)}")

## 10. Integration Guide

To integrate this validation into your live trading system:

```python
# In your signal generation code:
from app.comet.mcp_client import MCPClient
from app.comet.prompt_manager import PromptManager

def validate_signal_before_trade(signal_data):
    comet = MCPClient()
    pm = PromptManager()
    
    # Format prompt
    prompt = pm.format_prompt("signal_analysis", **signal_data)
    
    # Get Comet validation
    response = comet.query(prompt)
    analysis = response.get('content', '')
    
    # Extract confidence
    confidence = extract_confidence_score(analysis)
    
    # Decide whether to trade
    if confidence >= 7:
        return True, "High confidence"
    elif confidence >= 5:
        return True, "Medium confidence - reduce size"
    else:
        return False, "Low confidence - skip"
```